# Spatially explicit mapping of ecological similarity across a large

region using matching techniques.

Evan Muise [](https://orcid.org/0000-0003-3404-3220) (Department of Forest Resource Management, University of British Columbia, Vancouver, 2424 Main Mall, British Columbia, Canada)  
Nicholas Coops [](https://orcid.org/0000-0002-7859-8394) (Department of Forest Resource Management, University of British Columbia, Vancouver, 2424 Main Mall, British Columbia, Canada)  
June 6, 2024

ABSTRACT

In [ ]:
# include all libraries required by the R chunks here

library(terra)

terra 1.7.78

Loading required package: sf

Linking to GEOS 3.12.1, GDAL 3.8.4, PROJ 9.3.1; sf_use_s2() is TRUE

Support for Spatial objects (`sp`) was removed in {bcmaps} v2.0.0. Please use `sf` objects with {bcmaps}.

## 1 Introduction

A global biodiversity crisis is currently underway, driven by anthropogenic changes ([Dirzo and Raven 2003](#ref-dirzo2003)). Pressures such as climate change, land-use changes, and invasive species, are leading to species extinctions ([Thomas et al. 2004](#ref-thomas2004); [Urban 2015](#ref-urban2015)) and the homogenization of biological communities ([McGill et al. 2015](#ref-mcgill2015)). The Kunming-Montreal Global Biodiversity Framework (GBF) has been adopted as of December 2022, with the goal of restoring global biodiversity ([“Report of the Conference of the Parties to the Convention on Biological Diversity on the Second Part of Its Fifteenth Meeting” 2023](#ref-reporto2023)). Targets within this framework include restoring 30% of all degraded ecosystems, and protecting 30% of of Earth’s terrestrial, inland water, and marine areas ([“Report of the Conference of the Parties to the Convention on Biological Diversity on the Second Part of Its Fifteenth Meeting” 2023](#ref-reporto2023)). In the terrestrial environment, forest biomes have been shown to harbour the largest amount of biodiversity ([Myers 1988](#ref-myers1988); [Pimm and Raven 2000](#ref-pimm2000); [Cardinale et al. 2012](#ref-cardinale2012)), and provide key ecosystem services ([Thompson et al. 2009](#ref-thompson2009)). To provide these services, it is integral that these forest ecosystems are in good ecological condition, as represented by natural or near-natural levels of forest structure, forest function, and forest composition ([Marín et al. 2021](#ref-marín2021)).

While understanding forest condition is a key aspect of understanding biodiversity and the provision of ecosystem services due to their inherent linkages ([Marín et al. 2021](#ref-marín2021); [Cardinale et al. 2012](#ref-cardinale2012)), it is difficult to acquire suitable field-derived data across wide expanses of land due to the extreme financial and temporal costs associated with field campaigns. Remote sensing data, however, can provide a time-efficient, cost-effective alternative to field data, providing access to new spatially explicit and exhaustive datasets that can be linked to ecological condition, with additional metrics being proposed at a rapid pace ([Skidmore et al. 2021](#ref-skidmore2021); [Pereira et al. 2013](#ref-pereira2013); [Volker C. Radeloff et al. 2024](#ref-radeloff2024)). Advances in lidar technologies and modelling methods are allowing wall-to-wall estimates of forest stand structure to be generated across entire countries ([Matasci, Hermosilla, Wulder, White, Coops, Hobart, Bolton, et al. 2018](#ref-matasci2018); [Matasci, Hermosilla, Wulder, White, Coops, Hobart, and Zald 2018](#ref-matasci2018b); [Becker et al. 2023](#ref-becker2023)), serving as a finer scale indicator of ecosystem structure than the often previously used landscape fragmentation metrics ([Andrew, Wulder, and Coops 2012](#ref-andrew2012)). Productivity metrics have been used as a proxy for ecosystem function for many years ([Pettorelli et al. 2005](#ref-pettorelli2005), [2018](#ref-pettorelli2018)), with new Landsat derived datasets providing integrative annual estimates of energy availability at a 30 m spatial resolution([Volker C. Radeloff et al. 2024](#ref-radeloff2024); [V. C. Radeloff et al. 2019](#ref-radeloff2019); [Razenkova et al., n.d.](#ref-razenkova)). Remote sensing is quickly providing access to a plethora of datasets suitable monitoring the various facets of biodiversity and ecological condition ([Noss 1990](#ref-noss1990)). Using these datasets in conjunction with information on the location of known or assumed natural forests can allow researchers to identify similar forest stands or ecosystems, thus locating high quality forests across entire jurisdictions, even when they are facing anthropogenic pressures.

However, identifying natural forests is not a simple task. There are many definitions of what constitutes ‘naturalness’ in forest environments ([Winter 2012](#ref-winter2012)), and these difficulties are compounded by shifting baseline syndrome ([Pinnegar and Engelhard 2008](#ref-pinnegar2008)) and a lack of comprehensive historical data to generate reference states ([Balaguer et al. 2014](#ref-balaguer2014)). Other proposed methods to delineate baseline conditions include protected areas ([Arcese and Sinclair 1997](#ref-arcese1997)), and empirical estimates of the reference state, generated by modelling outcomes (oftentimes species abundances and occurrence) in the absence of anthropogenic disturbance ([Nielsen et al. 2007](#ref-nielsen2007)).

In this study, we seek to identify natural forests of good ecological condition at a 30 m spatial resolution across British Columbia, Canada. We first identify reference states for ecosystems across the province as protected areas with little-to-no anthropogenic disturbances. We then use an expanded coarsened exact matching (CEM) approach to compare the entire forested landscape of British Columbia to a reference state which accounts for forest type as well as topographic and climatic information. We expand the CEM technique to allow for the the spatially explicit reconstruction of natural ecosystem similarity, even when a perfect match is not available. We further examine how the coverage of natural forest areas varies between protected and unprotected areas in British Columbia. These methods will allow managers to identify regions in need of restoration, or that could be suitable for protection, and could be expanded to other regions.

## 2 Methods

### 2.1 Data

#### 2.1.1 Natural forests

We defined natural forests as forests under formal protection with little-to-no anthropogenic pressure. We obtained boundaries for the protected areas of British Columbia using the **wdpar** R package ([Hanson 2022](#ref-wdpar2022a) version 1.3.7), and filtered these protected areas larger than 100 ha and in the IUCN categories Ia, Ib, II, and IV Muise et al. ([2022](#ref-muise2022)). We use the Canadian Human Footprint generated by Hirsh-Pearson et al. ([2022](#ref-hirsh-pearson2022)), a 300 m estimating of anthropogenic pressures across the entirety of Canada, which accounts for pressures included in the global human footprint ([Venter et al. 2016](#ref-venter2016)), as well as pressures specific to the Canadian environment. The Canadian Human Footprint considers built environments, crop land, pasture land, human population density, nighttime lights, railways, roads, navigable waterways, dams and associated reservoirs, mining activity, oil and gas, and forestry pressures ([Hirsh-Pearson et al. 2022](#ref-hirsh-pearson2022)). We consider any pixel within our protected area criteria with a Canadian Human Footprint score below four as representative of natural ecosystems, which balances the tradeoff between having a suitable number of high-quality pixels, and low anthropogenic pressures being represented in the natural areas.

#### 2.1.2 Forest Ecosystem Structure

Wall-to-wall, 30 m forest structure metrics (canopy height, canopy cover, structural complexity, and aboveground biomass) were generated by Matasci, Hermosilla, Wulder, White, Coops, Hobart, Bolton, et al. ([2018](#ref-matasci2018)) for 2015 across the forested landscape of British Columbia. This data was generated by using a random forest-kNN approach, imputing airborne laser scanning derived forest structural attributes across the entirety of Canada using Landsat-derived best-available-pixel (BAP) composites ([White et al. 2014](#ref-white2014); [Hermosilla et al. 2016](#ref-hermosilla2016)) and topographic information ([Matasci, Hermosilla, Wulder, White, Coops, Hobart, Bolton, et al. 2018](#ref-matasci2018); [Matasci, Hermosilla, Wulder, White, Coops, Hobart, and Zald 2018](#ref-matasci2018b)). The BAP composites were derived by selecting optical observations from the Landsat archive (including Landsat-5 Thematic Mapper, Landsat-7 Enhanced Thematic Mapper Plus, and Landsat-8 Operational Land Imager imagery) over the course of the growing season, considering atmospheric effects (haze, clouds, cloud shadows) and distance from the desired composite date (in this case, August 31st). Details on the pixel scoring method can be found in White et al. ([2014](#ref-white2014)). The BAP composites were further refined by using a spectral trend analysis on the normalized burn ratio to remove noise, and missing pixels are infilled using temporally-interpolated values. This procedure results in gap-free, surface-reflectance image composites ([Hermosilla et al. 2015](#ref-hermosilla2015)).

#### 2.1.3 Forest Ecosystem Function

Landsat DHIs

#### 2.1.4 Auxiliary Datasets

In our matching procedure (see Section ) we use two core datasets as our covariates. Firstly, we use a 30 m digital elevation model and derived slope dataset from the Advanced Spaceborne Thermal Emission and Reflection Radiometer (ASTER) Version 2 GDEM product ([Tachikawa et al. 2011](#ref-tachikawa2011)). We also match on four climate variables; mean annual precipitation (MAP), mean annual temperature (MAT), mean warmest month temperature (MWMT), and mean coldest month temperature (MCMT) calculated from 1990-2020 climate normals using the ClimateNA software package at a 1 km spatial resolution, and downsampled to 30 m using cubic spline resampling in the **terra** R package ([Hijmans 2024](#ref-R-terra) version 1.7-71) in R ([R Core Team 2024](#ref-R-base) version 4.4.0).

### 2.2 Matching

To identify natural forests across the province, we implement an expanded coarsened exact matching (CEM) technique ([Iacus, King, and Porro 2012](#ref-iacus2012)). This allows us to assess the similarity of all forested pixels in the province to known natural forests, while controlling for covariates such as climate and topography. CEM creates comparable groups of observation among covariates by first coarsening the covariates into meaningful bins. In our case, we coarsen all six of our covariates into 10 quantile-based bins. As an example, elevation observations in the 0-10th percentile would be grouped into elevation bin one, the 10th-20th percentile of observations into bin two, and so on. Following the binning procedure, CEM performs exact matching on the coarsened covariates, and discards any unmatched samples. We diverge from this method by not dropping unmatched samples. Instead, we group our unmatched strata with their nearest neighbour in quartile space, up to a minimum of 30 samples, while minimizing the overall nearest neighbour distance. This method combines the advantages of CEM \[no bias checking, simple computation; Iacus, King, and Porro ([2012](#ref-iacus2012))\], with the ability to generate wall-to-wall comparisons of our treated cells to all other cells in the province.

### 2.3 Analysis

## 3 Results

## 4 Discussion

## 5 Conclusion

## References

Andrew, Margaret E., Michael A. Wulder, and Nicholas C. Coops. 2012. “Identification of *de Facto* Protected Areas in Boreal Canada.” *Biological Conservation* 146 (1): 97–107. <https://doi.org/10.1016/j.biocon.2011.11.029>.

Arcese, Peter, and A. R. E. Sinclair. 1997. “The Role of Protected Areas as Ecological Baselines.” *The Journal of Wildlife Management* 61 (3): 587–602. <https://doi.org/10.2307/3802167>.

Balaguer, Luis, Adrián Escudero, José F. Martín-Duque, Ignacio Mola, and James Aronson. 2014. “The Historical Reference in Restoration Ecology: Re-Defining a Cornerstone Concept.” *Biological Conservation* 176 (August): 12–20. <https://doi.org/10.1016/j.biocon.2014.05.007>.

Becker, Alexander, Stefania Russo, Stefano Puliti, Nico Lang, Konrad Schindler, and Jan Dirk Wegner. 2023. “Country-Wide Retrieval of Forest Structure from Optical and SAR Satellite Imagery with Deep Ensembles.” *ISPRS Journal of Photogrammetry and Remote Sensing* 195 (January): 269–86. <https://doi.org/10.1016/j.isprsjprs.2022.11.011>.

Bolton, Douglas K., Nicholas C. Coops, Txomin Hermosilla, Michael A. Wulder, Joanne C. White, and Colin J. Ferster. 2019. “Uncovering Regional Variability in Disturbance Trends Between Parks and Greater Park Ecosystems Across Canada (19852015).” *Scientific Reports* 9 (1): 1323. <https://doi.org/10.1038/s41598-018-37265-4>.

Cardinale, Bradley J., J. Emmett Duffy, Andrew Gonzalez, David U. Hooper, Charles Perrings, Patrick Venail, Anita Narwani, et al. 2012. “Biodiversity Loss and Its Impact on Humanity.” *Nature* 486 (7401): 59–67. <https://doi.org/10.1038/nature11148>.

Dirzo, Rodolfo, and Peter H. Raven. 2003. “Global State of Biodiversity and Loss.” *Annual Review of Environment and Resources* 28 (Volume 28, 2003): 137–67. <https://doi.org/10.1146/annurev.energy.28.050302.105532>.

Hanson, Jeffrey O. 2022. “Wdpar: Interface to the World Database on Protected Areas.” *Journal of Open Source Software* 7: 4594. <https://doi.org/10.21105/joss.04594>.

Hermosilla, Txomin, Michael A. Wulder, Joanne C. White, Nicholas C. Coops, and Geordie W. Hobart. 2015. “Regional Detection, Characterization, and Attribution of Annual Forest Change from 1984 to 2012 Using Landsat-Derived Time-Series Metrics.” *Remote Sensing of Environment* 170 (December): 121132. <https://doi.org/10.1016/j.rse.2015.09.004>.

Hermosilla, Txomin, Michael A. Wulder, Joanne C. White, Nicholas C. Coops, Geordie W. Hobart, and Lorraine B. Campbell. 2016. “Mass Data Processing of Time Series Landsat Imagery: Pixels to Data Products for Forest Monitoring.” *International Journal of Digital Earth* 9 (11): 10351054. <https://doi.org/10.1080/17538947.2016.1187673>.

Hijmans, Robert J. 2024. *Terra: Spatial Data Analysis*. <https://rspatial.org/>.

Hirsh-Pearson, Kristen, Chris J. Johnson, Richard Schuster, Roger D. Wheate, and Oscar Venter. 2022. “Canada’s Human Footprint Reveals Large Intact Areas Juxtaposed Against Areas Under Immense Anthropogenic Pressure.” *FACETS* 7 (January): 398–419. <https://doi.org/10.1139/facets-2021-0063>.

Iacus, Stefano M., Gary King, and Giuseppe Porro. 2012. “Causal Inference Without Balance Checking: Coarsened Exact Matching.” *Political Analysis* 20 (1): 1–24. <https://doi.org/10.1093/pan/mpr013>.

Marín, Ana Isabel, Dania Abdul Malak, Annemarie Bastrup-Birk, Gherardo Chirici, Anna Barbati, and Stefan Kleeschulte. 2021. “Mapping Forest Condition in Europe: Methodological Developments in Support to Forest Biodiversity Assessments.” *Ecological Indicators* 128 (September): 107839. <https://doi.org/10.1016/j.ecolind.2021.107839>.

Matasci, Giona, Txomin Hermosilla, Michael A. Wulder, Joanne C. White, Nicholas C. Coops, Geordie W. Hobart, Douglas K. Bolton, Piotr Tompalski, and Christopher W. Bater. 2018. “Three Decades of Forest Structural Dynamics over Canada’s Forested Ecosystems Using Landsat Time-Series and Lidar Plots.” *Remote Sensing of Environment* 216 (October): 697714. <https://doi.org/10.1016/j.rse.2018.07.024>.

Matasci, Giona, Txomin Hermosilla, Michael A. Wulder, Joanne C. White, Nicholas C. Coops, Geordie W. Hobart, and Harold S. J. Zald. 2018. “Large-Area Mapping of Canadian Boreal Forest Cover, Height, Biomass and Other Structural Attributes Using Landsat Composites and Lidar Plots.” *Remote Sensing of Environment* 209 (May): 90–106. <https://doi.org/10.1016/j.rse.2017.12.020>.

McGill, Brian J., Maria Dornelas, Nicholas J. Gotelli, and Anne E. Magurran. 2015. “Fifteen Forms of Biodiversity Trend in the Anthropocene.” *Trends in Ecology & Evolution* 30 (2): 104–13. <https://doi.org/10.1016/j.tree.2014.11.006>.

Muise, Evan R., Nicholas C. Coops, Txomin Hermosilla, and Stephen S. Ban. 2022. “Assessing Representation of Remote Sensing Derived Forest Structure and Land Cover Across a Network of Protected Areas.” *Ecological Applications* 32 (5): e2603. <https://doi.org/10.1002/eap.2603>.

Myers, Norman. 1988. “Threatened Biotas: "Hot Spots" in Tropical Forests.” *Environmentalist* 8 (3): 187–208. <https://doi.org/10.1007/BF02240252>.

Nielsen, S. E., E. M. Bayne, J. Schieck, J. Herbers, and S. Boutin. 2007. “A New Method to Estimate Species and Biodiversity Intactness Using Empirically Derived Reference Conditions.” *Biological Conservation* 137 (3): 403–14. <https://doi.org/10.1016/j.biocon.2007.02.024>.

Noss, Reed F. 1990. “Indicators for Monitoring Biodiversity: A Hierarchical Approach.” *Conservation Biology* 4 (4): 355–64. <https://doi.org/10.1111/j.1523-1739.1990.tb00309.x>.

Pereira, H. M., S. Ferrier, M. Walters, G. N. Geller, R. H. G. Jongman, R. J. Scholes, M. W. Bruford, et al. 2013. “Essential Biodiversity Variables.” *Science* 339 (6117): 277–78. <https://doi.org/10.1126/science.1229931>.

Pettorelli, Nathalie, Henrike Schulte to Bühne, Ayesha Tulloch, Grégoire Dubois, Cate Macinnis-Ng, Ana M. Queirós, David A. Keith, et al. 2018. “Satellite Remote Sensing of Ecosystem Functions: Opportunities, Challenges and Way Forward.” *Remote Sensing in Ecology and Conservation* 4 (2): 71–93. <https://doi.org/10.1002/rse2.59>.

Pettorelli, Nathalie, Jon Olav Vik, Atle Mysterud, Jean-Michel Gaillard, Compton J. Tucker, and Nils Chr. Stenseth. 2005. “Using the Satellite-Derived NDVI to Assess Ecological Responses to Environmental Change.” *Trends in Ecology & Evolution* 20 (9): 503–10. <https://doi.org/10.1016/j.tree.2005.05.011>.

Pimm, Stuart L., and Peter Raven. 2000. “Extinction by Numbers.” *Nature* 403 (6772): 843–45. <https://doi.org/10.1038/35002708>.

Pinnegar, John K., and Georg H. Engelhard. 2008. “The ‘Shifting Baseline’ Phenomenon: A Global Perspective.” *Reviews in Fish Biology and Fisheries* 18 (1): 1–16. <https://doi.org/10.1007/s11160-007-9058-6>.

R Core Team. 2024. *R: A Language and Environment for Statistical Computing*. Vienna, Austria: R Foundation for Statistical Computing. <https://www.R-project.org/>.

Radeloff, V. C., M. Dubinin, N. C. Coops, A. M. Allen, T. M. Brooks, M. K. Clayton, G. C. Costa, et al. 2019. “The Dynamic Habitat Indices (DHIs) from MODIS and Global Biodiversity.” *Remote Sensing of Environment* 222 (March): 204–14. <https://doi.org/10.1016/j.rse.2018.12.009>.

Radeloff, Volker C., David P. Roy, Michael A. Wulder, Martha Anderson, Bruce Cook, Christopher J. Crawford, Mark Friedl, et al. 2024. “Need and Vision for Global Medium-Resolution Landsat and Sentinel-2 Data Products.” *Remote Sensing of Environment* 300 (January): 113918. <https://doi.org/10.1016/j.rse.2023.113918>.

Razenkova, Elena, Katarzyna E Lewińska, He Yin, Laura S Farwell, Anna M Pidgeon, Patrick Hostert, Nicholas C Coops, and Volker C. Radeloff. n.d. “Medium-Resolution Dynamic Habitat Indices from Landsat Satellite Imagery.” *Remote Sensing of Environment*.

“Report of the Conference of the Parties to the Convention on Biological Diversity on the Second Part of Its Fifteenth Meeting.” 2023.

Skidmore, Andrew K., Nicholas C. Coops, Elnaz Neinavaz, Abebe Ali, Michael E. Schaepman, Marc Paganini, W. Daniel Kissling, et al. 2021. “Priority List of Biodiversity Metrics to Observe from Space.” *Nature Ecology & Evolution*, May. <https://doi.org/10.1038/s41559-021-01451-x>.

Tachikawa, Tetushi, Manabu Kaku, Akira Iwasaki, Dean B. Gesch, Michael J. Oimoen, Z. Zhang, Jeffrey J. Danielson, et al. 2011. “ASTER Global Digital Elevation Model Version 2 - Summary of Validation Results.” <http://pubs.er.usgs.gov/publication/70005960>.

Thomas, Chris D, Alison Cameron, Rhys E Green, Michel Bakkenes, Linda J Beaumont, Yvonne C Collingham, Barend F N Erasmus, et al. 2004. “Extinction Risk from Climate Change.” *Nature* 427: 5.

Thompson, Ian D., Brendan Mackey, Steven McNulty, and Alex Mosseler. 2009. *Forest Resilience, Biodiversity, and Climate Change: A Synthesis of the Biodiversity / Resiliende / Stability Relationship in Forest Ecosystems*. CBD Technical Series 43. Montreal: Secretariat of the Convention on Biological Diversity.

Urban, Mark C. 2015. “Accelerating Extinction Risk from Climate Change.” *Science* 348 (6234): 571–73. <https://doi.org/10.1126/science.aaa4984>.

Venter, Oscar, Eric W. Sanderson, Ainhoa Magrach, James R. Allan, Jutta Beher, Kendall R. Jones, Hugh P. Possingham, et al. 2016. “Sixteen Years of Change in the Global Terrestrial Human Footprint and Implications for Biodiversity Conservation.” *Nature Communications* 7 (1): 12558. <https://doi.org/10.1038/ncomms12558>.

White, Joanne. C., M. A. Wulder, G. W. Hobart, J. E. Luther, T. Hermosilla, P. Griffiths, N. C. Coops, et al. 2014. “Pixel-Based Image Compositing for Large-Area Dense Time Series Applications and Science.” *Canadian Journal of Remote Sensing* 40 (3): 192212. <https://doi.org/10.1080/07038992.2014.945827>.

Winter, S. 2012. “Forest Naturalness Assessment as a Component of Biodiversity Monitoring and Conservation Management.” *Forestry* 85 (2): 293–304. <https://doi.org/10.1093/forestry/cps004>.